# Large Language Models

Surely you've heard of ChatGPT. It's a large language model, which is a type of neural network used for natural language processing, amongst other things, based on the Transformer architecture.

We'll see two language models in this class:
- [Copilot](https://copilot.github.com), an AI pair programmer you may have already seen and can be active in VScode.
- [ChatGPT](http://chat.openai.com) or GPT-4 (GPT-4o), which is a large language model trained on a large corpus of text from the internet by OpenAI. Relative to Copilot, it generates larger blocks of reliable text but doesn't have quite the line-by-line pair programmer feeling of CoPilot.

# Copilot

Your AI pair programmer.

In this class, you may use Copilot primarily in two ways. 

The first is code completion. We'll begin with `fibonacci = ` and see what it gives.

If it did as expected in Jan. 2025, it just gave a list of Fibonacci numbers. Maybe this was what you wanted, but maybe it wasn't! What I wanted when I wrote this was a Fibonacci generating function. So let's try that.

In [ ]:
# What was generated another time
def fibonacci_generator_from_scratch(N):
    a, b = 0, 1
    for _ in range(N):
        a, b = b, a + b
    return a

We can also use comments to prompt Copilot. For example, `# compute pi as a series` will give us a series for computing pi.

# ChatGPT and GPT-4

To give you a sense of GPT, let's walk through an example. 

**Problem Statement:** Suppose I would like to arrange a set of 13 photos in a radial pattern, where each photo's position is set according to its overall brightness. The collage is then saved to a new image file. This will test the "skill" of our chatbot: can it accurately determine brightness, can it arrange $13$ images when $13$ doesn't divide nicely into a grid for a collage, and does it know what I mean by center?

You can find the conversation I had with GPT-4o [here](https://chatgpt.com/share/678ea155-320c-8013-948d-aa0bd36c3907). I'll show you the prompts and output here in case the link doesn't work in the future.

**Prompt:** write me a simple python code that combines 13 images of almost exactly the same size into a radial pattern where brighter photos appear nearer the center than darker photos

# Response 1
Here's a Python script that arranges 13 images in a radial pattern, placing brighter images closer to the center and darker images further outward. The script uses OpenCV (`cv2`) and NumPy to process images and `PIL` for brightness calculation. It assumes all images are nearly the same size.

### Steps:
1. Load the images and compute their brightness.
2. Sort images based on brightness.
3. Arrange them in a circular pattern with brighter images closer to the center.
4. Combine them into a single output image.

You'll need to install dependencies if you haven't:
```bash
pip install opencv-python numpy pillow
```

#### **Python Script:**
```python
import cv2
import numpy as np
from PIL import Image, ImageStat
import math
import glob

# Load images
image_paths = sorted(glob.glob("images/*.jpg"))  # Adjust path pattern if needed
images = [cv2.imread(img_path) for img_path in image_paths]

# Function to compute brightness
def compute_brightness(img):
    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    stat = ImageStat.Stat(pil_img)
    return stat.mean[0]  # Average brightness

# Compute brightness of each image
brightness_values = [compute_brightness(img) for img in images]

# Sort images based on brightness (brightest first)
sorted_images = [img for _, img in sorted(zip(brightness_values, images), reverse=True)]

# Define canvas size
img_h, img_w, _ = images[0].shape  # Assume all images are similar in size
canvas_size = (img_w * 5, img_h * 5)  # Adjust as needed
center = (canvas_size[0] // 2, canvas_size[1] // 2)

# Create blank canvas
canvas = np.zeros((canvas_size[1], canvas_size[0], 3), dtype=np.uint8)

# Arrange images radially
num_images = len(sorted_images)
radius_step = img_w // 2  # Distance increment
max_radius = radius_step * (num_images // 3)  # Control spread

for i, img in enumerate(sorted_images):
    angle = (i / num_images) * 2 * math.pi  # Spread images evenly
    radius = max_radius * (1 - (brightness_values[i] / max(brightness_values)))  # Brighter → closer

    x_offset = int(center[0] + radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + radius * math.sin(angle) - img_h // 2)

    # Place image on canvas
    canvas[y_offset:y_offset+img_h, x_offset:x_offset+img_w] = img

# Save output
cv2.imwrite("radial_collage.jpg", canvas)
cv2.imshow("Radial Pattern", canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()
```

### **How It Works:**
- It loads 13 images from an "images" folder.
- Computes brightness using PIL.
- Sorts them from brightest to darkest.
- Arranges them in a radial pattern, placing the brighter images closer to the center.

Let me know if you need modifications!

**Prompt:** It appears as though I don't have access to opencv-python dependencies. Can you suggest a rewrite that doesn't use this or cv2?

# Response 2
Certainly! Below is a revised version of your script that **does not use OpenCV (`cv2`)**. Instead, it relies entirely on **PIL (Pillow) and NumPy** to process and arrange the images in a radial pattern.

---

### **Python Script (No OpenCV Required)**
This script:
1. Loads images using **PIL (Pillow)**.
2. Computes brightness using **PIL**.
3. Sorts images from **brightest to darkest**.
4. Arranges them **radially**, with brighter images closer to the center.
5. Merges them into a **single output image**.

You may need to install dependencies:
```bash
pip install pillow numpy
```

---

```python
from PIL import Image, ImageStat
import numpy as np
import math
import glob

# Load images from a folder (adjust path as needed)
image_paths = sorted(glob.glob("images/*.jpg"))  # Change path if needed
images = [Image.open(img_path).convert("RGB") for img_path in image_paths]

# Function to compute brightness
def compute_brightness(img):
    stat = ImageStat.Stat(img)
    return stat.mean[0]  # Average brightness across the image

# Compute brightness for each image
brightness_values = [compute_brightness(img) for img in images]

# Sort images from brightest to darkest
sorted_images = [img for _, img in sorted(zip(brightness_values, images), reverse=True)]

# Define canvas size
img_w, img_h = images[0].size  # Assume all images are nearly same size
canvas_size = (img_w * 5, img_h * 5)  # Adjust as needed
center = (canvas_size[0] // 2, canvas_size[1] // 2)

# Create blank white canvas
canvas = Image.new("RGB", canvas_size, "white")

# Arrange images radially
num_images = len(sorted_images)
radius_step = img_w // 2  # Distance increment
max_radius = radius_step * (num_images // 3)  # Maximum spread radius

for i, img in enumerate(sorted_images):
    angle = (i / num_images) * 2 * math.pi  # Spread evenly in a circle
    radius = max_radius * (1 - (brightness_values[i] / max(brightness_values)))  # Brighter → closer

    x_offset = int(center[0] + radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + radius * math.sin(angle) - img_h // 2)

    # Paste image onto canvas
    canvas.paste(img, (x_offset, y_offset))

# Save and show the final image
canvas.save("radial_collage.jpg")
canvas.show()  # Opens the image for preview
```

---

### **How This Version Works:**
- Uses **PIL** (`Image` module) instead of OpenCV.
- Sorts images **based on brightness** using `ImageStat`.
- Places images **radially**, with brighter images **closer to the center**.
- Uses **NumPy and math** to calculate positions.

### **Output**
- Generates a `radial_collage.jpg` file in the working directory.
- Opens the image automatically using `canvas.show()`.

Let me know if you need any modifications!

In [1]:
from PIL import Image, ImageStat
import numpy as np
import math
import glob

# Load images from a folder (adjust path as needed)
image_paths = sorted(glob.glob("figures/*.jpg"))  # Changed path to figures, where I store the images
images = [Image.open(img_path).convert("RGB") for img_path in image_paths]

# Function to compute brightness
def compute_brightness(img):
    stat = ImageStat.Stat(img)
    return stat.mean[0]  # Average brightness across the image

# Compute brightness for each image
brightness_values = [compute_brightness(img) for img in images]

# Sort images from brightest to darkest
sorted_images = [img for _, img in sorted(zip(brightness_values, images), reverse=True)]

# Define canvas size
img_w, img_h = images[0].size  # Assume all images are nearly same size
canvas_size = (img_w * 5, img_h * 5)  # Adjust as needed
center = (canvas_size[0] // 2, canvas_size[1] // 2)

# Create blank white canvas
canvas = Image.new("RGB", canvas_size, "white")

# Arrange images radially
num_images = len(sorted_images)
radius_step = img_w // 2  # Distance increment
max_radius = radius_step * (num_images // 3)  # Maximum spread radius

for i, img in enumerate(sorted_images):
    angle = (i / num_images) * 2 * math.pi  # Spread evenly in a circle
    radius = max_radius * (1 - (brightness_values[i] / max(brightness_values)))  # Brighter → closer

    x_offset = int(center[0] + radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + radius * math.sin(angle) - img_h // 2)

    # Paste image onto canvas
    canvas.paste(img, (x_offset, y_offset))

# Save and show the final image
canvas.save("radial_collage.jpg")
canvas.show()  # Opens the image for preview

<img src="radial_collage.jpg">

Of course, this wasn't what I had in mind. Clearly, the sorting by brightness has worked, in a way. But we can refine.

**Prompt:** this works! But can you adjust the code so that no picture overlaps another and the result is as symmetrical as possible.

Great! Below is an improved version of the script that **ensures no images overlap** and distributes them **symmetrically** while keeping **brighter images closer to the center**.

### **Key Improvements:**
1. **No Overlaps** – Uses concentric circles, increasing the radius when needed.
2. **Symmetrical Placement** – Arranges images evenly along each circle.
3. **Brighter Images Closer to Center** – Keeps the brightness-based ordering.

---

### **Python Script (Overlap-Free & Symmetrical)**
```python
from PIL import Image, ImageStat
import numpy as np
import math
import glob

# Load images
image_paths = sorted(glob.glob("images/*.jpg"))  # Change folder path if needed
images = [Image.open(img_path).convert("RGB") for img_path in image_paths]

# Function to compute brightness
def compute_brightness(img):
    stat = ImageStat.Stat(img)
    return stat.mean[0]  # Average brightness

# Compute brightness of each image
brightness_values = [compute_brightness(img) for img in images]

# Sort images (brighter images first)
sorted_images = [img for _, img in sorted(zip(brightness_values, images), reverse=True)]

# Define canvas size
img_w, img_h = images[0].size  # Assume all images are similar in size
canvas_size = (img_w * 6, img_h * 6)  # Adjust based on number of images
center = (canvas_size[0] // 2, canvas_size[1] // 2)

# Create blank white canvas
canvas = Image.new("RGB", canvas_size, "white")

# Arrange images in a **non-overlapping radial pattern**
num_images = len(sorted_images)
layers = math.ceil(math.sqrt(num_images))  # Number of concentric circles
radius_step = int(1.5 * img_w)  # Distance increment per layer

image_positions = []
layer_count = [0] * layers  # Track images per layer

for i, img in enumerate(sorted_images):
    layer = i // layers  # Determine which concentric circle
    layer_radius = radius_step * (layer + 1)

    # Determine angle based on how many images are in this layer
    images_in_layer = max(6, layer * 6)  # Ensures a balanced distribution
    angle = (layer_count[layer] / images_in_layer) * 2 * math.pi
    layer_count[layer] += 1

    x_offset = int(center[0] + layer_radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + layer_radius * math.sin(angle) - img_h // 2)

    # Store positions to check for overlap
    image_positions.append((x_offset, y_offset))

    # Paste image onto canvas
    canvas.paste(img, (x_offset, y_offset))

# Save and show the final image
canvas.save("radial_collage_no_overlap.jpg")
canvas.show()  # Opens the image for preview
```

---

### **How This Works:**
- **Divides the images into layers** (concentric circles).
- **Ensures symmetry** by spacing images **evenly** per layer.
- **Expands the radius dynamically** based on layer count.
- **No Overlaps** by controlling the number of images per layer.

---

### **Expected Output:**
- The **brightest images are near the center**.
- Images are arranged in **balanced concentric circles**.
- **No overlapping** images.

This version ensures **better spacing and symmetry** while keeping the brightness-based ordering. 🚀 Let me know if you need any refinements!


In [3]:
from PIL import Image, ImageStat
import numpy as np
import math
import glob

# Load images
image_paths = sorted(glob.glob("figures/*.jpg"))  # Change folder path if needed
images = [Image.open(img_path).convert("RGB") for img_path in image_paths]

# Function to compute brightness
def compute_brightness(img):
    stat = ImageStat.Stat(img)
    return stat.mean[0]  # Average brightness

# Compute brightness of each image
brightness_values = [compute_brightness(img) for img in images]

# Sort images (brighter images first)
sorted_images = [img for _, img in sorted(zip(brightness_values, images), reverse=True)]

# Define canvas size
img_w, img_h = images[0].size  # Assume all images are similar in size
canvas_size = (img_w * 6, img_h * 6)  # Adjust based on number of images
center = (canvas_size[0] // 2, canvas_size[1] // 2)

# Create blank white canvas
canvas = Image.new("RGB", canvas_size, "white")

# Arrange images in a **non-overlapping radial pattern**
num_images = len(sorted_images)
layers = math.ceil(math.sqrt(num_images))  # Number of concentric circles
radius_step = int(1.5 * img_w)  # Distance increment per layer

image_positions = []
layer_count = [0] * layers  # Track images per layer

for i, img in enumerate(sorted_images):
    layer = i // layers  # Determine which concentric circle
    layer_radius = radius_step * (layer + 1)

    # Determine angle based on how many images are in this layer
    images_in_layer = max(6, layer * 6)  # Ensures a balanced distribution
    angle = (layer_count[layer] / images_in_layer) * 2 * math.pi
    layer_count[layer] += 1

    x_offset = int(center[0] + layer_radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + layer_radius * math.sin(angle) - img_h // 2)

    # Store positions to check for overlap
    image_positions.append((x_offset, y_offset))

    # Paste image onto canvas
    canvas.paste(img, (x_offset, y_offset))

# Save and show the final image
canvas.save("radial_collage_no_overlap.jpg")
canvas.show()  # Opens the image for preview


<img src ="radial_collage_no_overlap.jpg">

Better, but it does seem as though we lost photos!? And I did say symmetric, but was thinking radially symmetric.

**Prompt:** Sorry, I meant to make the final picture as radially symmetric as possible. And it appears that there is only 8 images in the collage, when there should be 13

Got it! Below is an improved version of the script that ensures **all 13 images are placed radially symmetrically**, without overlap, and still maintains the **brightness-based distance rule** (brighter images closer to the center).

---

### **Key Fixes & Improvements:**
1. **Ensures exactly 13 images are placed**.
2. **Perfect radial symmetry** by distributing images evenly along concentric circles.
3. **No overlaps** while keeping a natural circular arrangement.
4. **Brighter images closer to the center** with darker ones further out.

---

### **Revised Python Script**
```python
from PIL import Image, ImageStat
import numpy as np
import math
import glob

# Load images
image_paths = sorted(glob.glob("images/*.jpg"))  # Adjust path as needed
images = [Image.open(img_path).convert("RGB") for img_path in image_paths]

# Ensure exactly 13 images
if len(images) != 13:
    raise ValueError("This script requires exactly 13 images.")

# Function to compute brightness
def compute_brightness(img):
    stat = ImageStat.Stat(img)
    return stat.mean[0]  # Average brightness

# Compute brightness for each image
brightness_values = [compute_brightness(img) for img in images]

# Sort images (brighter images first)
sorted_images = [img for _, img in sorted(zip(brightness_values, images), reverse=True)]

# Define canvas size
img_w, img_h = images[0].size  # Assume all images are nearly the same size
canvas_size = (img_w * 6, img_h * 6)  # Adjust based on number of images
center = (canvas_size[0] // 2, canvas_size[1] // 2)

# Create blank white canvas
canvas = Image.new("RGB", canvas_size, "white")

# Radial arrangement: 1 center, 6 in inner ring, 6 in outer ring
positions = [(0, 0)]  # Start with center image
inner_radius = int(1.2 * img_w)  # Distance for the first ring
outer_radius = int(2.4 * img_w)  # Distance for the second ring

# Compute angles for positioning
angles_inner = np.linspace(0, 2 * np.pi, 7, endpoint=False)  # 6 evenly spaced
angles_outer = np.linspace(0, 2 * np.pi, 7, endpoint=False)  # 6 evenly spaced

# Compute positions (inner circle first, then outer)
for angle in angles_inner[1:]:  # Skip 0th element (center image)
    x_offset = int(center[0] + inner_radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + inner_radius * math.sin(angle) - img_h // 2)
    positions.append((x_offset, y_offset))

for angle in angles_outer[1:]:
    x_offset = int(center[0] + outer_radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + outer_radius * math.sin(angle) - img_h // 2)
    positions.append((x_offset, y_offset))

# Ensure we have exactly 13 positions
assert len(positions) == 13, "Incorrect number of positions computed!"

# Place images on the canvas
for img, (x, y) in zip(sorted_images, positions):
    canvas.paste(img, (x, y))

# Save and show the final image
canvas.save("radial_symmetric_collage.jpg")
canvas.show()  # Opens the image for preview
```

---

### **How This Works:**
1. **Arranges exactly 13 images symmetrically**:
   - **1 image in the center**.
   - **6 images in an inner circle** (medium brightness).
   - **6 images in an outer circle** (darker images).
2. **Spacing & alignment**:
   - Inner circle images are **closer to the center**.
   - Outer circle images are **further away**.
   - Images are **evenly spaced** around each ring.
3. **Ensures no overlapping** while maintaining a **radial symmetric pattern**.

---

### **Final Output:**
- **Brighter images** near the **center**.
- **Darker images** towards the **outer ring**.
- **Perfect radial symmetry** with **no overlaps**.

This should now create a **balanced, visually appealing radial collage**. 🚀 Let me know if you'd like any refinements!



In [5]:
from PIL import Image, ImageStat
import numpy as np
import math
import glob

# Load images
image_paths = sorted(glob.glob("figures/*.jpg"))  # Adjust path as needed
images = [Image.open(img_path).convert("RGB") for img_path in image_paths]

# Ensure exactly 13 images
if len(images) != 13:
    raise ValueError("This script requires exactly 13 images.")

# Function to compute brightness
def compute_brightness(img):
    stat = ImageStat.Stat(img)
    return stat.mean[0]  # Average brightness

# Compute brightness for each image
brightness_values = [compute_brightness(img) for img in images]

# Sort images (brighter images first)
sorted_images = [img for _, img in sorted(zip(brightness_values, images), reverse=True)]

# Define canvas size
img_w, img_h = images[0].size  # Assume all images are nearly the same size
canvas_size = (img_w * 6, img_h * 6)  # Adjust based on number of images
center = (canvas_size[0] // 2, canvas_size[1] // 2)

# Create blank white canvas
canvas = Image.new("RGB", canvas_size, "white")

# Radial arrangement: 1 center, 6 in inner ring, 6 in outer ring
positions = [(0, 0)]  # Start with center image
inner_radius = int(1.2 * img_w)  # Distance for the first ring
outer_radius = int(2.4 * img_w)  # Distance for the second ring

# Compute angles for positioning
angles_inner = np.linspace(0, 2 * np.pi, 7, endpoint=False)  # 6 evenly spaced
angles_outer = np.linspace(0, 2 * np.pi, 7, endpoint=False)  # 6 evenly spaced

# Compute positions (inner circle first, then outer)
for angle in angles_inner[1:]:  # Skip 0th element (center image)
    x_offset = int(center[0] + inner_radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + inner_radius * math.sin(angle) - img_h // 2)
    positions.append((x_offset, y_offset))

for angle in angles_outer[1:]:
    x_offset = int(center[0] + outer_radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + outer_radius * math.sin(angle) - img_h // 2)
    positions.append((x_offset, y_offset))

# Ensure we have exactly 13 positions
assert len(positions) == 13, "Incorrect number of positions computed!"

# Place images on the canvas
for img, (x, y) in zip(sorted_images, positions):
    canvas.paste(img, (x, y))

# Save and show the final image
canvas.save("radial_symmetric_collage.jpg")
canvas.show()  # Opens the image for preview


<img src ="radial_symmetric_collage.jpg">

This looks much better, but the brightest picture is not in the center. Mmmm, one more refinement.

**Prompt:** ok, that's looking much better, except for ONE thing: the brightest picture appears in the upper left corner and not the center. And the jpg files I want to combine are named A.jpeg, where A runs from 0 to 12; can you encode that?

Great! Below is the **final refined version** that ensures:
1. **The brightest image is placed at the center**.
2. **All 13 images (named `0.jpg` to `12.jpg`) are correctly loaded**.
3. **Radial symmetry is maintained**, with brightness controlling the placement.
4. **No overlaps** between images.

---

### **Updated Python Script**
```python
from PIL import Image, ImageStat
import numpy as np
import math
import glob

# Load images in order (ensuring names are "0.jpg" to "12.jpg")
image_paths = [f"images/{i}.jpg" for i in range(13)]  # Adjust folder path if needed
images = [Image.open(img_path).convert("RGB") for img_path in image_paths]

# Function to compute brightness
def compute_brightness(img):
    stat = ImageStat.Stat(img)
    return stat.mean[0]  # Average brightness

# Compute brightness for each image
brightness_values = [compute_brightness(img) for img in images]

# Sort images by brightness (brightest first)
sorted_indices = sorted(range(len(images)), key=lambda i: brightness_values[i], reverse=True)
sorted_images = [images[i] for i in sorted_indices]

# Define canvas size
img_w, img_h = images[0].size  # Assume all images are similar in size
canvas_size = (img_w * 6, img_h * 6)  # Adjust for spacing
center = (canvas_size[0] // 2, canvas_size[1] // 2)

# Create blank white canvas
canvas = Image.new("RGB", canvas_size, "white")

# Radial arrangement: 1 center, 6 in inner ring, 6 in outer ring
positions = [(center[0] - img_w // 2, center[1] - img_h // 2)]  # Start with center image
inner_radius = int(1.2 * img_w)  # Distance for the first ring
outer_radius = int(2.4 * img_w)  # Distance for the second ring

# Compute angles for positioning
angles_inner = np.linspace(0, 2 * np.pi, 7, endpoint=False)  # 6 evenly spaced
angles_outer = np.linspace(0, 2 * np.pi, 7, endpoint=False)  # 6 evenly spaced

# Compute positions (inner circle first, then outer)
for angle in angles_inner[1:]:  # Skip 0th element (center image)
    x_offset = int(center[0] + inner_radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + inner_radius * math.sin(angle) - img_h // 2)
    positions.append((x_offset, y_offset))

for angle in angles_outer[1:]:
    x_offset = int(center[0] + outer_radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + outer_radius * math.sin(angle) - img_h // 2)
    positions.append((x_offset, y_offset))

# Ensure we have exactly 13 positions
assert len(positions) == 13, "Incorrect number of positions computed!"

# Place images on the canvas (BRIGHTEST image is placed first at center)
for img, (x, y) in zip(sorted_images, positions):
    canvas.paste(img, (x, y))

# Save and show the final image
canvas.save("radial_symmetric_collage_correct.jpg")
canvas.show()  # Opens the image for preview
```

---

### **What’s Fixed?**
✅ **Brightest image now appears in the center.**  
✅ **Ensures all 13 images (`0.jpg` to `12.jpg`) are loaded correctly.**  
✅ **Maintains perfect radial symmetry** with inner and outer rings.  
✅ **No overlapping.**  

This will now produce a **perfectly structured radial pattern** where:
- **Brightest image (most white) is in the center**.
- **Next 6 brightest images form an inner ring**.
- **Darker images form an outer ring**.

🚀 Let me know if you need any refinements!

In [7]:
from PIL import Image, ImageStat
import numpy as np
import math
import glob

# Load images in order (ensuring names are "0.jpg" to "12.jpg")
image_paths = [f"figures/{i}.jpg" for i in range(13)]  # Adjust folder path if needed
images = [Image.open(img_path).convert("RGB") for img_path in image_paths]

# Function to compute brightness
def compute_brightness(img):
    stat = ImageStat.Stat(img)
    return stat.mean[0]  # Average brightness

# Compute brightness for each image
brightness_values = [compute_brightness(img) for img in images]

# Sort images by brightness (brightest first)
sorted_indices = sorted(range(len(images)), key=lambda i: brightness_values[i], reverse=True)
sorted_images = [images[i] for i in sorted_indices]

# Define canvas size
img_w, img_h = images[0].size  # Assume all images are similar in size
canvas_size = (img_w * 6, img_h * 6)  # Adjust for spacing
center = (canvas_size[0] // 2, canvas_size[1] // 2)

# Create blank white canvas
canvas = Image.new("RGB", canvas_size, "white")

# Radial arrangement: 1 center, 6 in inner ring, 6 in outer ring
positions = [(center[0] - img_w // 2, center[1] - img_h // 2)]  # Start with center image
inner_radius = int(1.2 * img_w)  # Distance for the first ring
outer_radius = int(2.4 * img_w)  # Distance for the second ring

# Compute angles for positioning
angles_inner = np.linspace(0, 2 * np.pi, 7, endpoint=False)  # 6 evenly spaced
angles_outer = np.linspace(0, 2 * np.pi, 7, endpoint=False)  # 6 evenly spaced

# Compute positions (inner circle first, then outer)
for angle in angles_inner[1:]:  # Skip 0th element (center image)
    x_offset = int(center[0] + inner_radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + inner_radius * math.sin(angle) - img_h // 2)
    positions.append((x_offset, y_offset))

for angle in angles_outer[1:]:
    x_offset = int(center[0] + outer_radius * math.cos(angle) - img_w // 2)
    y_offset = int(center[1] + outer_radius * math.sin(angle) - img_h // 2)
    positions.append((x_offset, y_offset))

# Ensure we have exactly 13 positions
assert len(positions) == 13, "Incorrect number of positions computed!"

# Place images on the canvas (BRIGHTEST image is placed first at center)
for img, (x, y) in zip(sorted_images, positions):
    canvas.paste(img, (x, y))

# Save and show the final image
canvas.save("radial_symmetric_collage_correct.jpg")
canvas.show()  # Opens the image for preview

<img src ="radial_symmetric_collage_correct.jpg">

### Caveats

This tech is really impressive. Questons for discussion:
- What about this problem and the dialog makes it a good / safe / sensible use of ChatGPT?
- What are use cases where you *wouldn't* want to use ChatGPT?

### How to think about this technology?

Roughly, the way I like to think about it is that GPT is good at interpolation between knowledge domains, but not extrapolation into new knowledge domains.

If you ask it a question you know is at the frontier of knowledge and ask it to push beyond, it generally won't do well.

I even have run into problems going beyond my own knowledge base. When trying to wrangle some rather messy data, I prompted ChatGPT for pandas code to help. However the code it returned included options in pandas functions that weren't real options. I guess this is not a problem in programming. Could it be a problem somewhere else?